In [2]:
import pandas as pd
import os

# Set the directory path where your TXT files are located
txt_files_directory = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\T-drive\\release\\taxi_log_2008_by_id'

# Set the directory path where you want to save the CSV files
csv_files_directory =  'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\dataset_preprocessed'
# Ensure the CSV directory exists
os.makedirs(csv_files_directory, exist_ok=True)

# Define the column names for the new CSV files
column_names = ['identifier', 'date_time', 'longitude', 'latitude']

# Iterate over each file in the TXT files directory
for filename in os.listdir(txt_files_directory):
    if filename.endswith('.txt'):
        txt_file_path = os.path.join(txt_files_directory, filename)
        csv_file_path = os.path.join(csv_files_directory, filename.replace('.txt', '.csv'))

        # Read the TXT file
        try:
            df = pd.read_csv(txt_file_path, header=None, names=column_names)

            # Save the DataFrame to CSV
            df.to_csv(csv_file_path, index=False)
            print(f"Converted {filename} to CSV.")
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

print("All TXT files have been converted to CSV.")

Converted 1.txt to CSV.
Converted 10.txt to CSV.
Converted 100.txt to CSV.
Converted 1000.txt to CSV.
Converted 10000.txt to CSV.
Converted 10001.txt to CSV.
Converted 10002.txt to CSV.
Converted 10003.txt to CSV.
Converted 10004.txt to CSV.
Converted 10005.txt to CSV.
Converted 10006.txt to CSV.
Converted 10007.txt to CSV.
Converted 10008.txt to CSV.
Converted 10009.txt to CSV.
Converted 1001.txt to CSV.
Converted 10010.txt to CSV.
Converted 10011.txt to CSV.
Converted 10012.txt to CSV.
Converted 10013.txt to CSV.
Converted 10014.txt to CSV.
Converted 10015.txt to CSV.
Converted 10016.txt to CSV.
Converted 10017.txt to CSV.
Converted 10018.txt to CSV.
Converted 10019.txt to CSV.
Converted 1002.txt to CSV.
Converted 10020.txt to CSV.
Converted 10021.txt to CSV.
Converted 10022.txt to CSV.
Converted 10023.txt to CSV.
Converted 10024.txt to CSV.
Converted 10025.txt to CSV.
Converted 10026.txt to CSV.
Converted 10027.txt to CSV.
Converted 10028.txt to CSV.
Converted 10029.txt to CSV.
Conv

In [7]:
import os
import random
import shutil

# Set the source and destination directories
source_dir = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\dataset_preprocessed'
destination_dir = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\dataset_preprocessed_subset'

# Ensure reproducibility by setting a random seed
random.seed(42)

# Get a list of all files in the source directory
all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

# Randomly select 500 files
selected_files = random.sample(all_files, 1000)

# Copy the selected files to the destination directory
for file in selected_files:
    source_path = os.path.join(source_dir, file)
    destination_path = os.path.join(destination_dir, file)
    shutil.copy2(source_path, destination_path)

print(f"Copied {len(selected_files)} files to {destination_dir}")


Copied 1000 files to C:\Users\ss6365\Desktop\Datasets\Tdirve\New


In [8]:
import os
import pandas as pd


# Input directory containing CSV files
input_directory = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\dataset_preprocessed_subset'

# Output directory where the merged CSV file will be saved
output_directory = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve'



# # List of important columns to keep
important_columns = ['identifier', 'date_time', 'longitude', 'latitude']  # Replace with your column names

# # Create a list to store dataframes from individual CSV files
dataframes = []

# # Iterate through CSV files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_directory, filename)
        df = pd.read_csv(file_path, usecols=important_columns)
        dataframes.append(df)

# # Concatenate dataframes vertically (along rows)
merged_df = pd.concat(dataframes, ignore_index=True)

# # Output file path for the merged CSV file
output_file_path = os.path.join(output_directory, 'merged_all_utility_subset.csv')

# # Save the merged dataframe to a CSV file
merged_df.to_csv(output_file_path, index=False)

print(f"Merged CSV file saved to {output_file_path}")

Merged CSV file saved to C:\Users\ss6365\Desktop\Datasets\Tdirve\merged_all_utility_subset.csv


In [9]:
df = pd.read_csv('C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\merged_all_utility_subset.csv')
df

,identifier,date_time,longitude,latitude
0,10002,2008-02-02 13:40:13,116.41106,39.92823
1,10002,2008-02-02 13:45:15,116.41104,39.92822
2,10002,2008-02-02 13:45:15,116.41104,39.92822
3,10002,2008-02-02 13:50:17,116.41085,39.92821
4,10002,2008-02-02 13:55:19,116.41098,39.92791
...,...,...,...,...
1507093,9753,2008-02-08 17:29:12,116.86800,40.37761
1507094,9753,2008-02-08 17:31:59,116.84535,40.37499
1507095,9753,2008-02-08 17:34:14,116.83732,40.37489
1507096,9753,2008-02-08 17:37:00,116.82555,40.37616


In [10]:

# Path to the CSV file
file_path = "C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\merged_all_utility_subset.csv"  # Update this path to your file's location

# Load the dataset
df = pd.read_csv(file_path)

# Convert 'time' column to datetime without specifying the format
df['time'] = pd.to_datetime(df['date_time'], errors='coerce')

# Calculate the minimum and maximum dates in the 'time' column
min_date = df['time'].min()
max_date = df['time'].max()

# Print the results
print(f"The earliest date of data logging is: {min_date}")
print(f"The latest date of data logging is: {max_date}")


The earliest date of data logging is: 2008-02-02 13:30:47
The latest date of data logging is: 2008-02-08 17:39:19


In [11]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Load your CSV file
csv_file_path = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\merged_all_utility_subset.csv'
df = pd.read_csv(csv_file_path)

# Assuming your columns are named 'Latitude' and 'Longitude'
latitude = 'latitude'
longitude = 'longitude'

# Create a map centered around the average location
map_center = [df[latitude].mean(), df[longitude].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Create a HeatMap layer and add it to the map
heat_data = [[row[latitude], row[longitude]] for index, row in df.iterrows()]
HeatMap(heat_data).add_to(m)

# Save the map to an HTML file
output_html = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\map_tdrive.html'
m.save(output_html)

print(f"Heatmap saved to {output_html}")

Heatmap saved to C:\Users\ss6365\Desktop\Datasets\Tdirve\map_tdrive.html


In [12]:
import os
import pandas as pd
import numpy as np

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees).
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def calculate_distance_for_file(csv_file):
    # Load the CSV file
    df = pd.read_csv(csv_file)
    
    # Calculate the distances
    distances = [
        haversine(lon1, lat1, lon2, lat2) 
        for lat1, lon1, lat2, lon2 in zip(df['latitude'][:-1], df['longitude'][:-1], df['latitude'][1:], df['longitude'][1:])
    ]
    
    # Sum the distances
    return sum(distances)

def calculate_cumulative_distance(directory):
    total_distance = 0
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            total_distance += calculate_distance_for_file(file_path)
    return total_distance

# Example usage
directory = 'C:\\Users\\ss6365\\Desktop\\Datasets\\Tdirve\\dataset_preprocessed_subset'  # Replace this with the path to your directory
cumulative_distance = calculate_cumulative_distance(directory)
print(f"Cumulative Distance Traversed: {cumulative_distance} km")

Cumulative Distance Traversed: 3792071.940243241 km
